### Modelagem de algoritmo: Gradiente Descente (GD)

Para a modelagem, adota-se as seguintes variáveis:

$$
\begin{aligned}
U_{1}        &= \frac{R_{1} - A^{2}}{4A^{2}} \\
U_{2}        &= \frac{R_{2} - A^{2}}{4A^{2}} \\
\overline{I}   &= \frac{I}{2A} \ e \ \overline{Q} = \frac{Q}{2A} \therefore \overline{Signal} = \overline{I} + j\overline{Q} = \frac{1}{2A}(I+jQ) = \frac{1}{2A}Signal   \\
\end{aligned}
$$

Com a mudança das variáveis, obtém-se as equações:

$$
\left\{\begin{matrix}
R_{1} = A^{2} + I^{2} + Q^{2} + 2AI \\
R_{2} = A^{2} + I^{2} + Q^{2} + 2AQ
\end{matrix}\right.
$$

$$
\left\{\begin{matrix}
R_{1} - A^{2} = I^{2} + Q^{2} + 2AI \ \ (\div \ 4A^{2}) \\
R_{2} - A^{2} = I^{2} + Q^{2} + 2AQ \ \ (\div \ 4A^{2})
\end{matrix}\right.
$$

$$
\left\{\begin{matrix}
U_{1} = \overline{I}^{2} + \overline{Q}^{2} + \overline{I} \\
U_{2} = \overline{I}^{2} + \overline{Q}^{2} + \overline{Q}
\end{matrix}\right.
$$

Ainda assim, para o sistema de equações encontrado, é definido as seguintes expressões:

$$
\left\{\begin{matrix}
X(\overline{I}, \overline{Q}) = \overline{I}^{2} + \overline{Q}^{2} + \overline{I} - U_{1} \Leftrightarrow X(\overline{I}, \overline{Q}) = 0 \\
Y(\overline{I}, \overline{Q}) = \overline{I}^{2} + \overline{Q}^{2} + \overline{Q} - U_{2} \Leftrightarrow Y(\overline{I}, \overline{Q}) = 0
\end{matrix}\right.
$$

Para resolver a expressão proposta pelo método, é necessário minimizar a seguinte função

$$
\begin{equation*}
G(\overline{I}, \overline{Q}) = X(\overline{I}, \overline{Q})^{2} + Y(\overline{I}, \overline{Q})^{2}
\end{equation*}
$$

Como estimativa inicial para a equação acima, definimos que:

$$
\begin{equation*}
\overline{I}_{(0)} = U_{1} - \frac{P}{4A^{2}}; \ \ \ \ \ \overline{Q}_{(0)} = U_{2} - \frac{P}{4A^{2}}
\end{equation*}
$$

Portanto, a atualização do gradiente da função é definida por

$$
\left\{\begin{matrix}
\overline{I}_{(n+1)} = \overline{I}_{(n)} - \mu \left[ X(\overline{I}_{(n)}, \overline{Q}_{(n)})(2\overline{I}_{(n)} + 1) + 2Y(\overline{I}_{(n)}, \overline{Q}_{(n)})\overline{I}_{(n)} \right] \\
\overline{Q}_{(n+1)} = \overline{Q}_{(n)} - \mu \left[ X(\overline{I}_{(n)}, \overline{Q}_{(n)})\overline{Q}_{(n)} + 2Y(\overline{I}_{(n)}, \overline{Q}_{(n)})(2\overline{Q}_{(n)} + 1) \right]
\end{matrix}\right.
$$

em que $\mu$ é o passo do algoritmo. O erro normalizado do algoritmo é dado por

$$
\begin{equation*}
\Delta \overline{V}_{(n)} = 2\sqrt{(\overline{I}_{(n)} - \overline{I})^{2} + (\overline{Q}_{(n)} - \overline{Q})^{2}}
\end{equation*}
$$


In [ ]:
from optic.metrics import signal_power
import numpy as np

def gradientDescent(R1, R2, sigWDM, sigLO, mu = 0.05, N=150):
    """
    Gradient Descent (GD)

    :param R1 and R2: ouput of SER [nparray]
    :param sigWDM: received signal [nparray]
    :param sigLO: local oscillator (LO) [nparray]
    :param mu: step size [scalar]
    :param N: number of iterations [scalar]
    
    :return sigOut: the inphase and quadrature components of the optical field [nparray]
    """
    
    A = sigLO                # Oscilador Local 
    P = signal_power(sigWDM) # Potência do sinal recebido
    
    U1 = (R1 - A**2) / (4*A**2) # Definindo U1 (Mudança de Variável)
    U2 = (R2 - A**2) / (4*A**2) # Definindo U2 (Mudança de Variável)
    
    overline_I = sigWDM.real / (2*A) # Define nova variável I
    overline_Q = sigWDM.imag / (2*A) # Define nova variável Q
    
    overline_I0 = U1 - P / (4*A**2) # Estimativa Inicial de I
    overline_Q0 = U2 - P / (4*A**2) # Estimativa Inicial de Q
    
    # Inicializando variáveis de iteração
    overline_In = overline_I0 # I(n) = I(0)
    overline_Qn = overline_Q0 # Q(n) = Q(0)
    
    # Iteração do algoritmo
    for nSteps in range(1, N): # Loop de N laços
               
        X_InQn = overline_In**2 + overline_Qn**2 + overline_In - U1 # Cálculo do valor de X(In,Qn)
        Y_InQn = overline_In**2 + overline_Qn**2 + overline_Qn - U2 # Cálculo do valor de Y(In,Qn)
        
        error = 2 * np.sqrt( (overline_In - overline_I)**2 + (overline_Qn - overline_Q)**2 ) # Cálculo do erro normalizado ▲V(n)
        
        gradientI = (X_InQn * (2*overline_In + 1) + 2 * Y_InQn * overline_In) # Cálculo do gradiente de I
        gradientQ = (X_InQn * overline_Qn + 2 * Y_InQn * (2*overline_Qn + 1)) # Cálculo do gradiente de Q
        
        overline_Inext = overline_In - mu * gradientI # Cálculo do valor de I(n+1)
        overline_Qnext = overline_Qn - mu * gradientQ # Cálculo do valor de Q(n+1)
        
        overline_In = overline_Inext # Atualiza I(n+1) para I(n)
        overline_Qn = overline_Qnext # Atualiza Q(n+1) para Q(n)
        
    overline_sigOut = (overline_In + 1j*overline_Qn) # estimation signal
    sigOut = (2*A) * overline_sigOut                 # recovered signal w/ SSBI cancellation
    
    return sigOut